In [2]:
import pandas as pd
import numpy as np


In [3]:
# func 모음

# 1. 종목코드를 6자리로 통일. ex) 990 -> 000990
def padding(x):
    x = str(x)
    return x.rjust(6, "0")

# 2. 결산월 12월인 것만 선택
# : 전체 데이터 4963개 중에서 12월 결산인 데이터가 4836개이다. 나머지 결산월의 데이터가 전체의 2.5%로 데이터량이 작기도하고 결측치가 많아 제거
def func_yearin(x):
    if x[5:7] == '12':
        return True
    else:
        return False

# 3. 회계년도컬럼, 년도만 뽑아서 int화
def change_year(x):
    return int(x[:4])

# 4. 연결, 개별재무제표 통합을 위해 연결재무 데이터 컬럼명 통일화 : "(IFRS연결)"제거
def func_replace_col1(x):
    x = x.replace(" ","")
    x = x.replace(",","")
    x = x.replace("(비교)","")
    x = x.replace("(IFRS)","")
    x = x.replace("(*)","")
    x = x.replace("[제조]","")
    return x.replace("(IFRS연결)","")

# 5. 연결, 개별재무제표 통합을 위해 개별재무 데이터 컬럼명 통일화 : "(IFRS)"제거
def func_replace_col2(x):
    x = x.replace(" ","")
    x = x.replace(",","")
    x = x.replace("(비교)","")
    x = x.replace("(IFRS)","")
    x = x.replace("(*)","")
    x = x.replace("[제조]","")
    return x.replace("(IFRS)","")

# 6. 2020년 raw data 제외
def func_rm_2020(x):
    if x[:4] == "2020":
        return False
    else:
        return True

# 7. True, False 반전
def func_not(x):
    return not x

In [4]:
배당_df = pd.read_csv('./datasets/raw/연결개별현금배당금.csv',encoding='cp949')
배당_df

,회사명,거래소코드,회계년도,배당금_현금
0,(주)CMG제약,58820,2011/12,NaN
1,(주)CMG제약,58820,2012/12,NaN
2,(주)CMG제약,58820,2013/12,NaN
3,(주)CMG제약,58820,2014/12,NaN
4,(주)CMG제약,58820,2015/12,NaN
...,...,...,...,...
22151,흥아해운(주),3280,2016/12,632567.0
22152,흥아해운(주),3280,2017/12,NaN
22153,흥아해운(주),3280,2018/12,NaN
22154,흥아해운(주),3280,2019/12,NaN


In [5]:
배당_df["거래소코드"] = 배당_df["거래소코드"].map(padding) # 거래소코드 padding
배당_df["회계년도"] = 배당_df["회계년도"].map(change_year) # 년도만 표시
배당_df

,회사명,거래소코드,회계년도,배당금_현금
0,(주)CMG제약,058820,2011,NaN
1,(주)CMG제약,058820,2012,NaN
2,(주)CMG제약,058820,2013,NaN
3,(주)CMG제약,058820,2014,NaN
4,(주)CMG제약,058820,2015,NaN
...,...,...,...,...
22151,흥아해운(주),003280,2016,632567.0
22152,흥아해운(주),003280,2017,NaN
22153,흥아해운(주),003280,2018,NaN
22154,흥아해운(주),003280,2019,NaN


In [6]:
배당_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22156 entries, 0 to 22155
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   회사명     22156 non-null  object 
 1   거래소코드   22156 non-null  object 
 2   회계년도    22156 non-null  int64  
 3   배당금_현금  12168 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 692.5+ KB


In [7]:
배당_df.isna().sum()

회사명          0
거래소코드        0
회계년도         0
배당금_현금    9988
dtype: int64

In [8]:
배당_df.fillna(0,inplace=True)

In [9]:
## 1년에 배당을 여러번 하는 기업이 존재 
## 1년에 한 배당금을 더해준다

배당_df = 배당_df.groupby(['회사명','회계년도','거래소코드'])[['배당금_현금']].sum().reset_index()


In [10]:
# 외국계 기업 제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

배당_df = 배당_df[배당_df["거래소코드"].map(func_isnt_fcompany)]

In [11]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(배당_df["회사명"].unique()))
print("거래소코드수 : ", len(배당_df["거래소코드"].unique()))

회사수 :  2487
거래소코드수 :  2515


In [12]:
# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (배당_df[배당_df["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(배당_df["회사명"].unique())


dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = 배당_df[배당_df["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
배당_df = pd.concat(dfs)
배당_df.reset_index(drop=True, inplace=True)

In [13]:
# 회사명이 2개인 거래소코드 존재 확인 - 엄슴
list_overlap_code = list()
for i in list(배당_df["거래소코드"].unique()):
    if len(배당_df[배당_df["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

['036420']

In [14]:
# 기업명 변경
배당_df[배당_df['거래소코드']=='036420']
배당_df.loc[배당_df['회사명']=='(주)제이콘텐트리',['회사명']] = '(주)콘텐트리중앙'

In [15]:
배당_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20642 entries, 0 to 20641
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   회사명     20642 non-null  object 
 1   회계년도    20642 non-null  int64  
 2   거래소코드   20642 non-null  object 
 3   배당금_현금  20642 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 645.2+ KB


In [16]:
배당_df

,회사명,회계년도,거래소코드,배당금_현금
0,(주)CMG제약,2011,058820,0.0
1,(주)CMG제약,2012,058820,0.0
2,(주)CMG제약,2013,058820,0.0
3,(주)CMG제약,2014,058820,0.0
4,(주)CMG제약,2015,058820,0.0
...,...,...,...,...
20637,흥아해운(주),2016,003280,632567.0
20638,흥아해운(주),2017,003280,0.0
20639,흥아해운(주),2018,003280,0.0
20640,흥아해운(주),2019,003280,0.0


In [17]:
## 배당을 한번도 안한 기업 제거
test = 배당_df.groupby('회사명')[['배당금_현금']].sum()
badang=test[test["배당금_현금"] != 0].index
배당_df= 배당_df[배당_df["회사명"].isin(badang)].reset_index(drop=True)


In [23]:
배당_df

,회사명,회계년도,거래소코드,배당금_현금
0,(주)DB하이텍,2011,000990,0.0
1,(주)DB하이텍,2012,000990,0.0
2,(주)DB하이텍,2013,000990,0.0
3,(주)DB하이텍,2014,000990,0.0
4,(주)DB하이텍,2015,000990,0.0
...,...,...,...,...
14777,흥아해운(주),2016,003280,632567.0
14778,흥아해운(주),2017,003280,0.0
14779,흥아해운(주),2018,003280,0.0
14780,흥아해운(주),2019,003280,0.0


In [39]:
배당_df['target'] = (배당_df.groupby('회사명')[['배당금_현금']].shift(0)!=0) &(배당_df.groupby('회사명')[['배당금_현금']].shift(1)!=0) & (배당_df.groupby('회사명')[['배당금_현금']].shift(2)!=0)

In [44]:
배당_df['target'] = 배당_df['target'].apply(lambda x : 1 if x==True else 0)

In [42]:
배당_df.drop('배당금_현금',axis=1,inplace=True)

In [46]:
배당_df.to_csv('target.csv',encoding='UTF-8-sig')